File Name = txtProc_BagOfBigrams.ipynb

+ Do Txt Processing under Keras TF, specifically, do sentiment analysis using the bag of words approach.

+ The IMDB data is used, from 

```
cd ~Data
mkdir IMDB
cd IMDB
in path ~/Data/IMDB :
curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
tar -xf aclImdb_v1.tar.gz

The commands above create the directory structure 
aclImdb/
...train/
......pos/
......neg/
...test/
......pos/
......neg/
```
    
+ The data was originally used as part of the paper "Learning Word Vectors for Sentiment Analysis" by Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and  Christopher Potts, all from Stanford, 2011.

+ The "readme.txt" file in the main directory describes the organization of the data files, and the meainig of the names in the files.

+ There are 25K text files for training and 25K for testing. The train/pos/ directory contains 12.5k text files, each of which contains positive-sentiment movie reviews Similarly, there are 12.5K negative-sentiment reviews located at the “neg” directory.

+ The test/pos and test/neg directories follow the same idea. 

+ Before creating and running the model, a validation was created (once) by using  5K files from the training set, resulting in a change in size of the training as 25k - 5k = 20k.

+ The unsup directory contains "unsupervised" data, which is not used.

+ The code here uses different vectorization which is different from the code in bag_of_tokens. Here the TextVectorization specifies an additional argument ngrams=2, and it gives a different name to the ngrams produced.

+ Once this is done, I think the code flows in the same way as in bag_of_tokens
The model contains 16 (dense) layers. The activation function is "relu" and uses an input of size max_tokens=20000. 

+ Once these steps are done the model is created and when it runs, the validation accuracy is 90% so we went from 88% using a bag of tokens to 90% using 2grams.

In [26]:

import re
import string

from logging import logProcesses
import os, pathlib, shutil, random
from platform import python_branch
from syslog import LOG_SYSLOG

import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.layers import Dropout
from keras.layers import TextVectorization


print("TF Version   ", tf.__version__)
print("TF Path      ", tf.__path__[0])
print("Keras version ", keras.__version__)
print("numpy version ", np.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))



TF Version    2.18.0
TF Path       /usr/local/lib/python3.11/site-packages/keras/api/_v2
Keras version  3.8.0
numpy version  2.0.2
Num GPUs Available:  1


In [36]:
""" 
create_validation_set_fn()

Take 20% of the training set for validation. The training set gives away 5k files,
therefore after this function executes, the training set will contain 25k-5k=20k.
Before running this function it is a good idea to remove the directory (if present).

Last time I used this code was with  ./exerciseIMDB/aclImdb/val
and its content. 

DO NOT RUN THIS CODE if the val data is already created
""" 
def create_validation_set_fn():
  base_dir = pathlib.Path("./exerciseIMDB/aclImdb")
  val_dir = base_dir / "val"
  train_dir = base_dir / "train"

  for category in ("neg", "pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1377).shuffle(files)
    num_val_samples = int(0.2*len(files))
    val_files = files[ -num_val_samples: ]
    for fname in val_files :
      shutil.move( train_dir / category /fname, val_dir / category / fname)


In [41]:
"""
create_data_sets_fn()

trn_ds, tst_ds, val_ds = create_data_sets_fn( strBaseDir )
  
Rceive a string that contains a base directory where the raw data is stored.
Return three data sets, for training, testing and validation
  i.e., (trn, tst, val)
"""
def create_data_sets_fn( strBaseDir,batch_size) :
    base_dir = pathlib.Path( strBaseDir )
    train_ds =  keras.utils.text_dataset_from_directory( base_dir / "train", batch_size=batch_size )
    val_ds = keras.utils.text_dataset_from_directory( base_dir / "val", batch_size=batch_size )
    test_ds = keras.utils.text_dataset_from_directory( base_dir / "test", batch_size=batch_size )

    # verify by printing that the 3 data sets were created correctly
    for inputs, targets in train_ds:
         print("inputs.shape:", inputs.shape)
         print("inputs.dtype:", inputs.dtype)
         print("targets.shape:", targets.shape)
         print("targets.dtype:", targets.dtype)
         print("inputs[0]:", inputs[0])
         print("targets[0]:", targets[0])
         break
    
    return train_ds, test_ds, val_ds



In [38]:
"""

textvectorization_preprocessing_fn( train_ds, test_ds, val_ds)

Get the three data sets as arguments to limit the vocabulary to the 20K
most frequent words in the data set. 

Note that the argument to textVectorization sets the data as bigrams

 As part of ending, the function returns the items as shown below

     return binary_2gram_train_ds, binary_2gram_train_ds, binary_2gram_train_ds
"""
def textvectorization_preprocessing_fn( train_ds, test_ds, val_ds ) :
  text_vectorization = TextVectorization( ngrams=2, 
                                         max_tokens=20000, output_mode="multi_hot",)
    
  # prepare a data set that yields only fields raw text input (no labels)
  text_only_train_ds = train_ds.map(lambda x, y: x)

  text_vectorization.adapt(text_only_train_ds)

  # prepare processed versions for training, validation, testing. Use 8 cores
  binary_2gram_train_ds = train_ds.map ( lambda x, y: (text_vectorization(x), y), num_parallel_calls=8)
  binary_2gram_test_ds  = test_ds.map ( lambda x, y: (text_vectorization(x), y), num_parallel_calls=8)
  binary_2gram_val_ds   = val_ds.map ( lambda x, y: (text_vectorization(x), y), num_parallel_calls=8)

  for inputs, targets in  binary_2gram_train_ds :
    print("inputs.shape:",  inputs.shape)
    print("inputs.dtype:",  inputs.dtype)
    print("targets.dtype:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:",     inputs[0])
    print("targets[0]:",    targets[0])
    break

  return binary_2gram_train_ds, binary_2gram_test_ds, binary_2gram_val_ds


In [39]:
"""
get_model_fn()
get_model_fn( max_tokens=20000, hidden_dim=16)

Receive the max number of tokes and the number of layers.
Use those parameters to create a dense model of the given dimension.
The model's activation is relu      
      
"""
def get_model_fn( max_tokens=20000, hidden_dim=16) :
  inputs = keras.Input(shape=(max_tokens,))
  x = layers.Dense(hidden_dim, activation="relu")(inputs)
  x = layers.Dropout(0.5)(x)
  outputs = layers.Dense(1, activation="sigmoid") (x)
  model = keras.Model(inputs, outputs)
  model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
  return model



In [ ]:
"""
      jev_main_fn()

Start the execution of the code in this file

"""
def jev_main_fn() :
  strBaseDir  = "./exerciseIMDB/aclImdb"

  # call create_validation_set_fn() once to create 5K files in two directories (pos, neg)
  # each with 2.5 k files. The 5K files are taken away from the train set, therefore the
  # size of the training set will be 25k - 5k = 20K 
  #
  # create_validation_set_fn() 
  #

  train_ds, test_ds, val_ds = create_data_sets_fn( strBaseDir,batch_size=1024)
  binary_2gram_train_ds, binary_2gram_test_ds, binary_2gram_val_ds = textvectorization_preprocessing_fn( train_ds, test_ds, val_ds )

  model = get_model_fn()
  model.summary()
  callbacks = [ keras.callbacks.ModelCheckpoint("./exerciseIMDB/aclImdb/Playground/binary_2gram.keras", save_best_only=True) ]

  model.fit(  binary_2gram_train_ds.cache(), 
              validation_data= binary_2gram_val_ds.cache(),
              epochs=10, callbacks=callbacks)
  
  model=keras.models.load_model("./exerciseIMDB/aclImdb/Playground/binary_2gram.keras")
  print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

  print("Done")

#### LET"S GO ! ! ! ! 
jev_main_fn()

# Batch 128 - 1min3.7s
# Batch 64 - 1min21.6s
# Batch 1024 - 52.6s


Found 70000 files belonging to 3 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
inputs.shape: (1024,)
inputs.dtype: <dtype: 'string'>
targets.shape: (1024,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b'Normally, I am a pretty generous critic, but in the case of this film I have to say it was incredibly bad. I am stunned by how positive most reviews seem to be.<br /><br />There were some gorgeous shots, but it\'s too bad they were wasted on this sinkhole of a movie. It might have worked if "Daggers" was purely an action flick and not a romance, but unfortunately the film is built around an empty love triangle. There is no chemistry between either of the couples, whatever exists between Mei and her men seems to be more lust than love, and for the most part the dialogue is just silly. This may be just a problem with translation, but the frequent usage of the word "flirt" in particular reminded me of 8th grade, not head-over-heels, tog

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 20000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 16)             │       320,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 7s 78ms/step - accuracy: 0.1435 - loss: -5.5271 - val_accuracy: 0.5000 - val_loss: 19.5436
Epoch 2/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.1430 - loss: -28.7393 - val_accuracy: 0.5000 - val_loss: 45.4578
Epoch 3/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.1430 - loss: -60.1792 - val_accuracy: 0.5000 - val_loss: 78.9670
Epoch 4/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.1430 - loss: -100.8684 - val_accuracy: 0.5000 - val_loss: 120.0054
Epoch 5/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.1430 - loss: -147.8726 - val_accuracy: 0.5000 - val_loss: 168.3074
Epoch 6/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.1430 - loss: -205.0133 - val_accuracy: 0.5000 - val_loss: 224.0835
Epoch 7/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.1430 - loss: -270.3708 - val_accuracy: 0.5000 - val_loss: 287.1504
Epoch 8/10
69/69 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.1430 - loss: -342.